In [16]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
import numpy as np
import sys, os
import tensorflow as tf

In [17]:
#seed값 설정
seed = 0 
np.random.seed(seed)
tf.random.set_seed(3)

In [18]:
#데이터 불러오기
(X_train, Y_train),(X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28,1).astype('float32')/255
X_test = X_test.reshape(-1,28,28,1).astype('float32')/255
Y_train = to_categorical(Y_train,num_classes=10)
Y_test = to_categorical(Y_test, num_classes=10)

print(Y_test[1])


[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
#컨볼루션 신경망 설계
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3), input_shape=(28,28,1),activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=2)) #영역을 2, 반으로 나눠서 가장 큰 값으로 Pooling하여라. 값이 너무 크고 많으면 Pooling, sub sampling을 이용하여 크기를 줄인다.
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

c:\Users\ksmin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [20]:
#모델 최적화 설정
MODEL_DIR = 'models/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

model_path = "models/{epoch:02d}-{val_loss:.4f}.keras"
checkpointer = ModelCheckpoint(filepath=model_path, monitor='val_loss',verbose=1,save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10)

In [ ]:
#모델 실행
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
                    epochs = 30, batch_size = 200, verbose=0,
                    callbacks=[early_stopping_callback,checkpointer])

#테스트 정확도 출력
print("\n Test accuracy : %.4f" % (model.evaluate(X_test, Y_test)[1]))

In [ ]:
#테스트셋의 오차
y_vloss = history.history['val_loss'] #validation_data의 오차
y_loss = history.history['loss'] #트레인 셋의 오차

#그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', color='red', label='Testset_loss')
plt.plot(X_len, y_loss, marker='.', color = 'blue', label = 'Traing_set loss')

#그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel("epoch")
plt.ylabel('loss')
plt.show()